In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv('data/WNCAATourneyDetailedResults.csv')
print(df.shape)
df.head()

(567, 34)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,138,3124,69,3201,55,N,0,28,57,...,34,3,5,17,19,12,18,4,1,18
1,2010,138,3173,67,3395,66,N,0,23,59,...,27,14,15,18,26,8,8,8,6,22
2,2010,138,3181,72,3214,37,H,0,26,57,...,15,3,8,10,21,4,16,6,4,20
3,2010,138,3199,75,3256,61,H,0,25,63,...,20,17,22,16,21,13,16,5,4,24
4,2010,138,3207,62,3265,42,N,0,24,68,...,26,11,17,16,22,9,10,3,4,12


### Feature Engineering

In [56]:
# All Regular Season and Post Season Games
reg_season = pd.read_csv('data/WRegularSeasonDetailedResults.csv')
post_season = pd.read_csv('data/WNCAATourneyDetailedResults.csv')

# Combine data
all_games = pd.concat([reg_season, post_season],
                      axis=0,
                      ignore_index=True)

print(all_games.shape)
all_games.tail()

(52149, 34)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
52144,2018,147,3163,94,3376,65,N,0,37,63,...,7,8,13,9,18,10,13,5,1,12
52145,2018,147,3323,84,3332,74,N,0,31,72,...,15,14,18,6,23,11,5,8,3,18
52146,2018,151,3280,73,3257,63,N,1,25,62,...,20,5,7,16,22,7,11,7,0,22
52147,2018,151,3323,91,3163,89,N,1,33,70,...,24,4,6,16,25,20,17,5,6,18
52148,2018,153,3323,61,3280,58,N,0,22,52,...,16,10,17,12,26,9,15,8,2,21


In [57]:
# Field goal percentage
all_games['WFG%'] = all_games['WFGM'] / all_games['WFGA']
all_games['LFG%'] = all_games['LFGM'] / all_games['LFGA']

# 3-Point field goal percentage
all_games['WFG3%'] = all_games['WFGM3'] / all_games['WFGA3']
all_games['LFG3%'] = all_games['LFGM3'] / all_games['LFGA3']

# Free throw field goal percentage
all_games['WFT%'] = all_games['WFTM'] / all_games['WFTA']
all_games['LFT%'] = all_games['LFTM'] / all_games['LFTA']

# Spread
all_games['spread'] = all_games['WScore'] - all_games['LScore']

all_games.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LStl,LBlk,LPF,WFG%,LFG%,WFG3%,LFG3%,WFT%,LFT%,spread
0,2010,11,3103,63,3237,49,H,0,23,54,...,7,6,19,0.425926,0.370370,0.555556,0.230769,0.631579,0.600000,14
1,2010,11,3104,73,3399,68,N,0,26,62,...,4,2,27,0.419355,0.396825,0.416667,0.190476,0.571429,0.518519,5
2,2010,11,3110,71,3224,59,A,0,29,62,...,6,0,15,0.467742,0.327586,0.400000,0.142857,0.583333,0.826087,12
3,2010,11,3111,63,3267,58,A,0,27,52,...,14,5,14,0.519231,0.243243,0.363636,0.230769,0.555556,0.640000,5
4,2010,11,3119,74,3447,70,H,1,30,74,...,4,2,14,0.405405,0.337838,0.350000,0.529412,0.636364,0.523810,4


- I'm going to create a linear regression model based on the spread.
- I will create a dataset where the set of features are swapped between fields so it's agnostic to which field is for the winning team
- The predicted spread for the actual tournament games will be based off the average value of these fields for the teams to be playing
- The spread will be calculated and the probability will be back calculated from that spread

### Create training data

In [58]:
# Isolate features of interest
columns = ['WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'WFG%', 'WFG3%', 'WFT%',
           'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'LFG%', 'LFG3%', 'LFT%',
           'spread']
isolate1 = all_games[columns]

# Create mirror of dataframe but with opposite spread
columns = ['LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'LFG%', 'LFG3%', 'LFT%',
           'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'WFG%', 'WFG3%', 'WFT%',
           'spread']
isolate2 = all_games[columns]
isolate2['spread'] = isolate2['spread'] * -1

# Have same column names for dataframe (doesn't really matter the column name)
isolate2.columns = isolate1.columns
all_games_2 = pd.concat([isolate1, isolate2],
                        axis=0,
                        ignore_index=True)
print(all_games_2.shape)
all_games_2.tail()

(104298, 21)


/Users/jad/miniconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,WFG%,WFG3%,WFT%,...,LDR,LAst,LTO,LStl,LBlk,LPF,LFG%,LFG3%,LFT%,spread
104293,9,18,10,13,5,1,12,0.500000,0.142857,0.615385,...,21,25,7,7,3,12,0.587302,0.600000,1.000000,-29
104294,6,23,11,5,8,3,18,0.430769,0.266667,0.777778,...,32,21,10,2,2,13,0.430556,0.300000,0.904762,-10
104295,16,22,7,11,7,0,22,0.385714,0.200000,0.714286,...,28,9,15,4,4,14,0.403226,0.266667,0.760000,-10
104296,16,25,20,17,5,6,18,0.493506,0.375000,0.666667,...,23,14,15,7,3,11,0.471429,0.315789,0.826087,-2
104297,12,26,9,15,8,2,21,0.372881,0.250000,0.588235,...,29,8,17,8,1,17,0.423077,0.222222,0.882353,-3


In [60]:
# Check for any null values
# classification_df[~pd.isnull(classification_df).any(axis=1)]
all_games_2[pd.isnull(all_games_2).any(axis=1)]

# 150 rows with an NaN, will replace with 0
all_games_2.fillna(value=0, inplace=True)

# Check
all_games_2[pd.isnull(all_games_2).any(axis=1)]

,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,WFG%,WFG3%,WFT%,...,LDR,LAst,LTO,LStl,LBlk,LPF,LFG%,LFG3%,LFT%,spread


In [61]:
# Data for train and test
X = all_games_2.iloc[:, 0:20].values
y = all_games_2.iloc[:, 20].values

### Build models

In [89]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'    # To precent dead kernel
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import KFold

In [90]:
def cross_validation(model):
    # Subset data
    X = all_games_2.iloc[:, 0:20].values
    y = all_games_2.iloc[:, 20].values

    # K-fold CV
    kf = KFold(n_splits=5, random_state=42, shuffle=True)

    r2_scores = []
    mae_scores = []
    for train_index, test_index, in kf.split(X):
        # Train
        train_x, train_y = X[train_index], y[train_index]
        reg = model
        reg.fit(train_x, train_y)

        # Test
        test_x, test_y = X[test_index], y[test_index]
        score = reg.score(test_x, test_y)
        r2_scores.append(score)
        
        # Get MAE
        pred = reg1.predict(test_x)
        MAE = np.sum(np.abs(pred - test_y))
        mae_scores.append(MAE)
    
    print("R^2:", np.mean(r2_scores))
    print("MAE:", np.mean(mae_scores))
    print("--------------")

In [93]:
# Models
reg1 = LinearRegression()
reg2 = XGBRegressor()

In [94]:
# 5-fold CV
cross_validation(reg1)
cross_validation(reg2)

R^2: 0.955858433581119
MAE: 62103.32080330777
--------------
R^2: 0.9309365973968571
MAE: 62098.53783775109
--------------


In [95]:
# Fit final models
reg1.fit(X, y)
reg2.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

### Create tournament data features

In [100]:
sub = pd.read_csv('WSampleSubmissionStage2.csv')

# Split ID to get TEAM IDs
sub['Year'], sub['Team1ID'], sub['Team2ID'] = sub['ID'].str.split('_').str
sub[['Year', 'Team1ID', 'Team2ID']] = sub[['Year', 'Team1ID', 'Team2ID']].apply(pd.to_numeric)
print(sub.shape)
sub.head()

(2016, 5)


,ID,Pred,Year,Team1ID,Team2ID
0,2019_3101_3113,0.5,2019,3101,3113
1,2019_3101_3114,0.5,2019,3101,3114
2,2019_3101_3120,0.5,2019,3101,3120
3,2019_3101_3124,0.5,2019,3101,3124
4,2019_3101_3125,0.5,2019,3101,3125


In [101]:
# Get all teamID's in tournament
all_ids = set()

for element1, element2 in zip(sub['Team1ID'], sub['Team2ID']):
    all_ids.add(element1)
    all_ids.add(element2)

64

In [106]:
all_games_2019 = all_games[all_games['Season'] == 2019]
print(all_games_2019.shape)
all_games_2019.head()

(5240, 41)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LStl,LBlk,LPF,WFG%,LFG%,WFG3%,LFG3%,WFT%,LFT%,spread
46342,2019,1,3113,81,3230,43,H,0,29,65,...,12,1,21,0.446154,0.257576,0.357143,0.176471,0.565217,0.400000,38
46343,2019,1,3120,97,3212,48,H,0,38,71,...,10,0,20,0.535211,0.258621,0.321429,0.200000,0.705882,0.750000,49
46344,2019,1,3124,100,3311,39,H,0,42,69,...,4,1,21,0.608696,0.226415,0.538462,0.210526,0.450000,0.687500,61
46345,2019,1,3135,68,3148,63,H,0,21,64,...,5,2,29,0.328125,0.380952,0.105263,0.200000,0.631579,0.625000,5
46346,2019,1,3136,58,3262,57,A,0,22,51,...,11,4,23,0.431373,0.410714,0.375000,0.375000,0.421053,0.727273,1


In [173]:
columns = ['TeamID', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'WFG%', 'WFG3%', 'WFT%']
all_team_features = pd.DataFrame(columns=columns)

# Create feature vector for each team in tournament and make master table
for team in all_ids:

    # Find all games for team
    team_games = all_games_2019[(all_games_2019['WTeamID'] == team) |
                                 (all_games_2019['LTeamID'] == team)]

    # Keep only the last 15 games as core statistics
    last_15 = team_games.iloc[-15:,:]

    # Isolate their winning and losing games
    winning_games = last_15[last_15['WTeamID'] == team]
    losing_games = last_15[last_15['LTeamID'] == team]

    # Get essential row data
    columns1 = ['WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']
    columns2 = ['LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']
    winning_games = winning_games[columns1]
    losing_games = losing_games[columns2]

    # Rename columns arbitrarily
    losing_games.columns = winning_games.columns

    # Stack rows
    last_15 = pd.concat([winning_games, losing_games],
                        axis=0,
                        ignore_index=True)

    # Get sums of columns
    last_15_sums = last_15.sum()

    # Create final stats
    columns = ['TeamID', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'WFG%', 'WFG3%', 'WFT%']
    team_features = pd.DataFrame([np.zeros(11)], columns = columns)
    team_features['TeamID'] = team
    team_features['WFG%'] = last_15_sums['WFGM'] / last_15_sums['WFGA']
    team_features['WFG3%'] = last_15_sums['WFGM3'] / last_15_sums['WFGA3']
    team_features['WFT%'] = last_15_sums['WFTM'] / last_15_sums['WFTA']
    team_features['WOR'] = last_15_sums['WOR'] / 15
    team_features['WDR'] = last_15_sums['WDR'] / 15
    team_features['WAst'] = last_15_sums['WAst'] / 15
    team_features['WTO'] = last_15_sums['WTO'] / 15
    team_features['WStl'] = last_15_sums['WStl'] / 15
    team_features['WBlk'] = last_15_sums['WBlk'] / 15
    team_features['WPF'] = last_15_sums['WPF'] / 15
    
    all_team_features = pd.concat([all_team_features, team_features],
                                  axis=0,
                                  ignore_index=True)

In [228]:
sub.iloc[18,:]

ID         2019_3101_3200
Pred                  0.5
Year                 2019
Team1ID              3101
Team2ID              3200
Name: 18, dtype: object

In [258]:
def round_of_rating(number):
    """
    Round a number to the closest half integer.
    >>> round_of_rating(1.3)
    1.5
    >>> round_of_rating(2.6)
    2.5
    >>> round_of_rating(3.0)
    3.0
    >>> round_of_rating(4.1)
    4.0
    """
    return round(float(number) * 2) / 2

In [259]:
import math

def create_submission_probabilities(model, file):

    spread_probabilities = pd.read_csv('the_spread.csv')

    game_probabilites = []

    # Get probabilities for all games
    for i in range(len(sub)):
        team1id = sub.loc[i, 'Team1ID']
        team2id = sub.loc[i, 'Team2ID']

        # Get Team Data
        team1 = all_team_features[all_team_features['TeamID'] == team1id]
        team2 = all_team_features[all_team_features['TeamID'] == team2id]

        # Drop non-essential info
        team1 = team1.drop(columns=['TeamID'])
        team2 = team2.drop(columns=['TeamID'])

        # Reset index
        team1.reset_index(inplace=True, drop=True)
        team2.reset_index(inplace=True, drop=True)

        # Need to create two feature vectors
        rename_columns = {'WOR': 'LOR', 
                          'WDR': 'LDR', 
                          'WAst': 'LAst', 
                          'WTO': 'LTO', 
                          'WStl': 'LStl',
                          'WBlk': 'LBlk',
                          'WPF': 'LPF', 
                          'WFG%': 'LFG%',
                          'WFG3%': 'LFG3%', 
                          'WFT%': 'LFT%'}
        team2_renamed = team2.rename(columns=rename_columns)
        first_vector = pd.concat([team1, team2_renamed], axis=1, ignore_index=True).values

        team1_renamed = team1.rename(columns=rename_columns)
        second_vector = pd.concat([team2, team1_renamed], axis=1, ignore_index=True).values

        # Understand which side of the spread the team is on
        # Found that we only need one of the values so just pred1
        pred1 = model.predict(first_vector)
        pred2 = model.predict(second_vector)

        # Determine probability from spread value
        pred1 = round_of_rating(pred1)             # Setup for lookup of probability

        probability = 0

        if pred1 > 15.5:
            probability = 0.9999

        elif pred1 < -15.5:
            probability = 0.0001

        else:
            probability = float(spread_probabilities[spread_probabilities['Spread'] == pred1]['Probability'])

        game_probabilites.append(probability)

    # Create submission
    final_submission = sub.copy()
    final_submission['Pred'] = game_probabilites
    final_submission = final_submission[['ID', 'Pred']]
    final_submission.to_csv('submissions/' + file, index=False)

In [260]:
create_submission_probabilities(reg1, 'LR.csv')
create_submission_probabilities(reg2, 'XGBoost.csv')

In [257]:
sub.shape

(2016, 5)

### Overide with top seeds
- 1 seeds: 
    - Baylor: 3124 - 3101 - 3314, 3143
    - Mississippi: 3280 - 3380 - 3377, 3155
    - Louisville: 3257 - 3352 - 3276, 3243
    - Notre Dame: 3323 - 3126 - 3277, 3141
- 2 seeds:
    - Iowa: 3234 - 3273
    - Oregon: 3332 - 3340
    - UConn: 3163 - 3406
    - Stanford: 3390 - 3413
- Overide first rounds with: 0.9999

In [209]:
rounded

-4.0

,Spread,Probability
0,15.5,0.922718
1,15.0,0.916818
2,14.5,0.910510
3,14.0,0.903774
4,13.5,0.896589


In [233]:
spread_probabilities.dtypes

Spread         float64
Probability    float64
dtype: object

In [201]:
pred2

array([3.94252124])

In [184]:
all_games = pd.DataFrame(columns=['Team1_Pyth', 'Team1_Luck','Team1_SOS_Pyth', 'Team1_NCSOS_Pyth',
                                  'Team2_Pyth', 'Team2_Luck','Team2_SOS_Pyth', 'Team2_NCSOS_Pyth'])

for i in range(len(sub)):
    print(i)
    # Create features for each game
    team1 = sub.loc[i, 'Team1ID']
    team2 = sub.loc[i, 'Team2ID']

    team1_data = team_codes_pomeroy[team_codes_pomeroy['TeamID'] == team1]
    team2_data = team_codes_pomeroy[team_codes_pomeroy['TeamID'] == team2]
    
    # Drop columns (required for model input)
    team1_data.drop(columns=['TeamID', 'Team', 'Year'], inplace=True)
    team2_data.drop(columns=['TeamID', 'Team', 'Year'], inplace=True)
    
    # Reset index
    team1_data.reset_index(inplace=True, drop=True)
    team2_data.reset_index(inplace=True, drop=True)

    # Rename columns (required for model input)
    team1_data = team1_data.rename(columns={'Pyth': 'Team1_Pyth',
                                       'Luck': 'Team1_Luck',
                                       'SOS Pyth': 'Team1_SOS_Pyth',
                                       'NCSOS Pyth': 'Team1_NCSOS_Pyth'})
    team2_data = team2_data.rename(columns={'Pyth': 'Team2_Pyth',
                                       'Luck': 'Team2_Luck',
                                       'SOS Pyth': 'Team2_SOS_Pyth',
                                       'NCSOS Pyth': 'Team2_NCSOS_Pyth'})

    # Merge Data
    single_game = pd.concat([team1_data, team2_data], axis=1)
    
    # Add to master dataframe
    all_games = pd.concat([all_games, single_game],
                                   ignore_index=True)

,TeamID,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,WFG%,WFG3%,WFT%
17,3113,12.066667,24.533333,15.2,11.6,4.866667,2.933333,16.0,0.425234,0.309434,0.675926


In [185]:
# Feature vector column requirement
all_games_2.columns

Index(['WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'WFG%', 'WFG3%',
       'WFT%', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'LFG%',
       'LFG3%', 'LFT%', 'spread'],
      dtype='object')

In [ ]:
# Create Submission files
submission1 = pd.read_csv('SampleSubmissionStage2.csv')
submission2 = pd.read_csv('SampleSubmissionStage2.csv')

# Add in predictions
submission1['Pred'] = pred1_prob
submission2['Pred'] = pred2_prob

# Write submissions
submission1.to_csv('submissions/LR.csv', index=False)
submission2.to_csv('submissions/XGBoost.csv', index=False)